# <FONT COLOR="RED">***Overview***</FONT>

---
---


Descripción de la sección.

# <FONT COLOR="ORANGE">**Prepare Notebook**</FONT>

---
---



It's important to note that the notebook is designed to run from **Google Colab** (with free GPU T4 resources), so some packages used aren't installed, as they're included by default in the environment. If you're having issues with any of the packages, we recommend checking your installed versions to install the missing packages in their supported versions.

In [1]:
# Color messages function
def print_mess(mess:str, color="green") -> None:
  match color:
    case "green":
      print(f"\033[92m{mess}\033[0m")
    case "red":
      print(f"\033[91m{mess}\033[0m")
    case "yellow":
      print(f"\033[93m{mess}\033[0m")
    case "white":
      print(f"\033[97m{mess}\033[0m")


In [2]:
# Identify environment keys
import os
import subprocess

# Google Colaboratory environment flag
is_in_Google_Colab = True

if "COLAB_" not in "".join(os.environ.keys()):
  is_in_Google_Colab = False
  print_mess("Notebook is actually running out of Google Colaboratory environment", "yellow")
  print_mess("Installing necessary packages...", "white")
  try:
    # Install unsloth first with its dependencies
    subprocess.run([
        "pip", "install",
        "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    ])
    subprocess.run([
        "pip", "install", "--no-deps"
        "xformers", "trl<=0.9.0", "peft", "accelerate", "bitsandbytes"
    ])
    subprocess.run([
        "pip", "install",
        "datasets>=3.4.1,<4.0.0", "tqdm",
    ])
    print_mess("Packages installed successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")
else:
  print_mess("Notebook is running in Google Colaboratory environment", "green")
  print_mess("Installing necessary packages...", "white")
  try:
    # Install unsloth first with its dependencies
    subprocess.run([
        "pip", "install",
        "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    ])
    subprocess.run([
        "pip", "install", "--no-deps"
        "xformers==0.0.29.post3", "trl<=0.9.0", "peft", "accelerate", "bitsandbytes"
    ])

    # Install other packages
    subprocess.run([
        "pip", "install", "--upgrade"
        "google-api-python-client", "google-auth-httplib2", "google-auth-oauthlib"
    ])

    subprocess.run([
        "pip", "install",
        "datasets>=3.4.1,<4.0.0", "tqdm"
    ])

    subprocess.run([
        "pip", "install",
        "sentencepiece", "protobuf", "huggingface_hub", "hf_transfer"
    ])

    print_mess("Packages installed successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")

Notebook is running in Google Colaboratory environment
Installing necessary packages...
Packages installed successfully


In [3]:
# Fine-tune packages
from unsloth import FastLanguageModel, is_bfloat16_supported, unsloth_train # This import need ve first
import numpy as np
import time
import traceback
import torch
from transformers import TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback
from trl import SFTTrainer, SFTConfig

# Sava model packages
from datetime import datetime

# Nootebook visual presentation packages
from tqdm import tqdm

# Dataset packages
import io
import json
from datasets import Dataset
from typing import Union
import re
from unsloth.chat_templates import get_chat_template

# Google authentication packages only used in Google Colaboratory environment
if is_in_Google_Colab:
  from google.colab import auth
  from googleapiclient.errors import HttpError
  from googleapiclient.discovery import build
  from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload, MediaFileUpload

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: Unsloth: Please install unsloth_zoo via `pip install unsloth_zoo`

If you're using **Google Collaboratory** to run this notebook and don't have the model files, we recommend mounting the **Google Drive** through the authentication cell.

If you **don't want to allow access to your Google Drive**, we invite you to visit our **[GitHub repository](https://)** where you can find the notebook that train the model and there you can save the model files in your local storage.

In [ ]:
# Google connection and authentication
if is_in_Google_Colab:
  auth.authenticate_user()
  drive_service = build("drive", "v3")

Likewise, if you're working in an **environment outside of Google Colaboratory** or have chosen **not to mount your Google Drive** to the Google Colaboratory environment, you must have the model files in the same directory for use with this notebook.

In [ ]:
# Here you must put the name of your folder name that allocated the model files
main_model_folder_name = "Model_Versions"

# Here you must put the name of your files with the JSON extension when notebook
# is outside of Google Colaboratory
eval_dataset_name = "RMI_Intentions_Evaluation_Dataset.json"

In [ ]:
# Search folder function
def search_folder(id_folder:str, name_folder:str) -> dict:
  folder_list = drive_service.files().list(
      q=f"'{id_folder}' in parents and mimeType = 'application/vnd.google-apps.folder' and name contains '{name_folder}'",
      spaces="drive",
      fields="nextPageToken, files(id, name, createdTime)",
      orderBy="createdTime desc"
  ).execute()

  return folder_list

In [ ]:
# If notebook is executed from Google Colaboratory, and you want to use the last
# version of the model trained files, it's necessary that mount the Google Drive
# unit and execute the following cells.

if is_in_Google_Colab:
  main_model_folder_id = "1giaO-m_gID4LImM1x4AoyebrsfW6DLYY"
  main_model_folder_name = "Translation_Intent_Model_"
  main_dataset_folder_id = "1liZABLNR-wodEouxbAaqTvYdrtkOKuKx"
  main_dataset_folder_name = "RMI_Intentions_Dataset_"

  # Search the most recent created folder
  model_folder_list = search_folder(main_model_folder_id, main_model_folder_name)

  # Search the most recent created folder
  dataset_folder_list = search_folder(main_dataset_folder_id, main_dataset_folder_name)

  # Obtain the most recent folder id of the model folder
  if not model_folder_list.get("files", []):
    print_mess("No folders found", "red")
  else:
    model_folder_id = model_folder_list.get("files", [])[0]["id"]
    print_mess(f"The most recent model folder ID is: {model_folder_id}")

  # Obtain the most recent folder id of the dataset folder
  if not dataset_folder_list.get("files", []):
    print_mess("No folders found", "red")
  else:
    dataset_folder_id = dataset_folder_list.get("files", [])[0]["id"]
    print_mess(f"The most recent dataset folder ID is: {dataset_folder_id}")

In [ ]:
# Create the local directory to download the model files if it doesn't exist
local_download_path = "/content/Q-LoRa_Phi_3_Model"
os.makedirs(local_download_path, exist_ok=True)

In [ ]:
# Search and save the model files
if is_in_Google_Colab:
  id_model_file_found = []
  name_model_file_found = []

  model_file_list = drive_service.files().list(
      q = f"'{model_folder_id}' in parents",
      spaces = "drive",
      fields = "nextPageToken, files(id, name)"
  ).execute()

  # Identify file in the file list
  if not model_file_list.get("files", []):
    print_mess("No files found", "red")
  else:
    id_model_file_found = [file["id"] for file in model_file_list.get("files", [])]
    name_model_file_found = [file["name"] for file in model_file_list.get("files", [])]
    for i in range(len(id_model_file_found)):
      print_mess(f"The file '{name_model_file_found[i]}' was found with ID: {id_model_file_found[i]}")

In [ ]:
# Download function
def download_and_save_file(file_id:str, file_name:str, local_path:str, model_download:bool) -> Union[str, None]:
  # Request to download file
  request=drive_service.files().get_media(fileId=file_id)
  fh = io.BytesIO()
  downloader = MediaIoBaseDownload(fh, request)
  done = False

  # Download process
  with tqdm(total = 100, unit = "%", desc = f"Downloading {file_name} file") as pbar:
    while done is False:
      status, done = downloader.next_chunk()
      pbar.update(int(status.progress() * 100) - pbar.n)

  if model_download:
    # Save process
    with open(os.path.join(local_path, file_name), "wb") as f:
      f.write(fh.getvalue())
    return None
  else:
    # Move the cursor to the beginning for correct reading of the file
    fh.seek(0)
    return fh.read().decode("utf-8")

In [ ]:
# Download model files in the Google Colab environment
if is_in_Google_Colab:
  try:
    print_mess("Downloading model files...", "green")
    for i in range(len(id_model_file_found)):
      print_mess(f"Downloading {name_model_file_found[i]}...", "white")
      download_and_save_file(id_model_file_found[i], name_model_file_found[i], local_download_path, True)
    print_mess("\nModel files downloaded successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")
else:
  try:
    print_mess("Reading model files...", "green")
    for i in range(len(id_model_file_found)):
      print_mess(f"Reading {name_model_file_found[i]}...", "white")
      with open(name_model_file_found[i], "r") as f:
        file_content = f.read()
        with open(os.path.join(local_download_path, name_model_file_found[i]), "w") as outfile:
          outfile.write(file_content)
      print_mess(f"{name_model_file_found[i]} saved", "white")
    print_mess("Model files read successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")

Likewise, if you're working in an **environment outside of Google Colaboratory** or have chosen **not to mount your Google Drive** to the Google Colaboratory environment, you must have the evaluation dataset file in the same directory for use with this notebook.

In [ ]:
# Find datasets inside the Google Drive folder
if is_in_Google_Colab:
  dataset_list = drive_service.files().list(
      q = f"'{dataset_folder_id}' in parents and name = '{eval_dataset_name}'",
      spaces = "drive",
      fields = "nextPageToken, files(id, name)"
  ).execute()

  # Identify item in the result file list
  if not dataset_list.get("files", []):
    print_mess(f"The file '{eval_dataset_name}' was not found in the provided folder", "red")
  else:
    id_evaluation_dataset = dataset_list.get("files", [])[0]["id"]
    print_mess(f"The file '{eval_dataset_name}' was found with ID: {id_evaluation_dataset}")

In [ ]:
# Download txt file in the Google Colaboratoy environment
if is_in_Google_Colab:
  try:
    print_mess("Downloading evaluation datasets...", "green")
    eval_json_dataset = json.loads(download_and_save_file(id_evaluation_dataset, eval_dataset_name, "", False))
    print_mess("\nDataset downloaded successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")
    print(e)
else:
  try:
    print_mess("Reading evaluation datasets...", "green")
    with open(eval_dataset_name, "r") as f:
      eval_json_dataset = json.loads(f.read())
    print_mess("Dataset read successfully", "green")
  except Exception as e:
    print_mess("Something went wrong", "red")

# <FONT COLOR="ORANGE">**Download Trained Model**</FONT>

---
---


For this project, **we chose to use the Phi-3 model**, specifically its **small language model (SLM) version called "Phi-3-mini-4k-instruct"**, which offers great capabilities as a conversational model and in code generation while consuming minimal resources.

In this section, the previously trained and downloaded model is loaded into the Google Colab environment.

In [ ]:
# Download Model Parameters
model_name = "Q-LoRa_Phi_3_Model"
sequence_length = 4096
quantization_model = True
seed = 3407

In [ ]:
# Download Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length=sequence_length,
    dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16,
    load_in_4bit = quantization_model
)

FastLanguageModel.for_inference(model)

# <FONT COLOR="ORANGE">**Dataset Preprocessing**</FONT>

---
---


To further **ensure the model correctly understands the training data**, it is important to provide it with said data in a correct format. In this case, since **Phi-3 is a conversational model that uses special tokens to delimit information**, it is important to make conscious use of these tokens, which are:

*   **<|user|>**: Indicates where the input provided by a user begins, i.e., what the model is asked to do.
*   **<|assistant|>**: Indicates where the output it should generate as a conversational assistant model begins, i.e., during training, it teaches it how to respond.
*   **<|end|>**: Indicates to the model where either of the two previous sections ends. This token is essential for the model to understand where what the user says ends and where the assistant should respond.

Seeking to achieve the best possible quality in the responses generated by the model, **we added a custom token to each of the conversations from which the model will learn**, which is:

*   **<|system_message|>**: Through this token, we apply the Prompt Engineering Zero-Shot technique during training to further specialize the model's behavior and obtain the most accurate responses possible.


In [ ]:
# Create prompt template
tokenizer = get_chat_template(
  tokenizer,
  chat_template = 'phi-3',
  mapping = {
    'role': 'from',
    'content': 'value',
    'system': 'system_message',
    'user': 'user_operator',
    'assistant': 'model_answer'
  }
)

In [ ]:
# Drop the model answer
eval_json_dataset_no_answer = eval_json_dataset
for conversation in eval_json_dataset_no_answer['conversations']:
  for message in conversation:
    if message['from'] == 'model_answer':
      conversation.remove(message)
      break

# <FONT COLOR="ORANGE">**Model Execution**</FONT>

---
---


Descripción de la sección

In [ ]:
# Execution Parameters
generated_outputs = []
generation_times = [] # Lista para almacenar los tiempos de generación

for conversation in eval_json_dataset_no_answer["conversations"]:
  # Generate tensors
  inputs = tokenizer.apply_chat_template(
      conversation,
      tokenize = True,
      add_generation_prompt = True,
      return_tensors = "pt"
  ).to("cuda")

  # Start time measurement
  start_time = time.time()

  # Generate Output
  generated_output = model.generate(
      input_ids = inputs,
      max_new_tokens = sequence_length/2,
      use_cache = True
  )

  # End time measurement and store
  end_time = time.time()
  generation_times.append(end_time - start_time)

  # Decode output to add in list
  decoded_output = tokenizer.decode(generated_output[0], skip_special_tokens=True)
  generated_outputs.append(decoded_output)

print_mess("Generation times:", "green")
print(generation_times)

In [ ]:
generated_outputs[0]


In [ ]:
generated_outputs[1]

In [ ]:

generated_outputs[2]

In [ ]:

generated_outputs[3]

In [ ]:
generated_outputs[4]

In [ ]:
generated_outputs[5]

In [ ]:
generated_outputs[6]

In [ ]:
generated_outputs[7]

In [ ]:
generated_outputs[8]

In [ ]:
generated_outputs[9]

In [ ]:
generated_outputs[10]

In [ ]:
generated_outputs[11]

In [ ]:
generated_outputs[12]

In [ ]:
generated_outputs[13]

In [ ]:
generated_outputs[14]

In [ ]:
# Result processing
tmf_format_outputs = []
for output in generated_outputs:
  # Regular expression to find text within triple backticks
  pattern = re.search(r"```(.*?)```", output, re.DOTALL)
  if pattern:
    # Extract the matched text and remove leading/trailing whitespace
    extracted_text = pattern.group(1).strip()
    tmf_format_outputs.append(extracted_text)
  else:
    # If no match is found, It append a empty string to identify errors in generation
    tmf_format_outputs.append("")

print_mess("Results processed successfully", "green")

# <FONT COLOR="ORANGE">**Save Results**</FONT>

---
---


In [ ]:
# Get today's date
now = datetime.now()
today = now.strftime("%Y-%m-%d_%H-%M-%S")

# Define the folder ID where the new folder will be created
folder_id = "1U9L29UTG59gRVHVy46-gvqhLH5TFjaRf"

# Define the name of the new folder
new_folder_name = f"Model_Results_{today}"

In [ ]:
# Create new folder
try:
  file_metadata = {
      "name": new_folder_name,
      "parents": [folder_id],
      "mimeType": "application/vnd.google-apps.folder"
  }
  folder = drive_service.files().create(
      body = file_metadata,
      fields = "id"
  ).execute()
  created_folder_id = folder.get("id")
  print_mess(f"Folder '{new_folder_name}' created successfully with ID: {created_folder_id}", "green")
except HttpError as error:
  print_mess(f"An error occurred: {error}", "red")
  created_folder_id = None

In [ ]:
# Define file name
save_file_name = "generated_tmf_format_outputs.json"

# Convert the list to a JSON string
json_generated_outputs = json.dumps(tmf_format_outputs, indent=2)

# Create file content as bytes
save_file_content = json_generated_outputs.encode("utf-8")

In [ ]:
# Define save file metadata
save_file_metadata = {
    "name": save_file_name,
    "parents": [created_folder_id],
    "mimeType": "application/json"
}

In [ ]:
# Create MediaIoBaseUpload object
media = MediaIoBaseUpload(
    io.BytesIO(save_file_content),
    mimetype = "application/json",
    resumable = True
)

In [ ]:
# Upload file
try:
  uploaded_file = drive_service.files().create(
      body = save_file_metadata,
      media_body = media,
      fields = "id"
  ).execute()
  print_mess(f"File '{save_file_name}' uploaded successfully with ID: {uploaded_file.get('id')}", "green")
except HttpError as error:
  print_mess(f"An error occurred: {error}", "red")

In [2]:
# ============================================================================
# CÓDIGO SIMPLIFICADO: CALCULAR MÉTRICAS CARGANDO DESDE GOOGLE DRIVE
# ============================================================================

import json
import numpy as np
import subprocess
import os
import io

# Instalar paquetes necesarios para las métricas
try:
    subprocess.run(["pip", "install", "scikit-learn", "nltk", "sacrebleu", "matplotlib"], check=True, capture_output=True)
    print("Packages installed successfully")
except Exception as e:
    print(f"Error installing packages: {e}")

# Imports para métricas
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import sacrebleu
from difflib import SequenceMatcher

# Google Drive imports
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError

# Descargar recursos de NLTK
nltk.download('punkt', quiet=True)

# ============================================================================
# 1. AUTENTICAR Y CONFIGURAR GOOGLE DRIVE
# ============================================================================

print("Authenticating with Google Drive...")
auth.authenticate_user()
drive_service = build('drive', 'v3')
print("✓ Authenticated successfully\n")

# ============================================================================
# 2. FUNCIÓN PARA DESCARGAR ARCHIVOS DESDE GOOGLE DRIVE
# ============================================================================

def download_file_from_drive(file_id, file_name):
    """Descarga un archivo de Google Drive y retorna su contenido como string."""
    try:
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.BytesIO()
        downloader = MediaIoBaseDownload(fh, request)
        done = False

        print(f"Downloading {file_name}...", end=" ")
        while done is False:
            status, done = downloader.next_chunk()

        # Mover cursor al inicio y decodificar
        fh.seek(0)
        content = fh.read().decode("utf-8")
        print("✓")
        return content

    except HttpError as error:
        print(f"✗ Error: {error}")
        return None

# ============================================================================
# 3. DESCARGAR ARCHIVOS CON LOS IDs CORRECTOS
# ============================================================================

# IDs de los archivos
generated_outputs_file_id = "1CQx7tABaPHvVrUQgPiy2TIoV60CFsqL0"  # generated_tmf_format_outputs.json
eval_dataset_file_id = "1-3umMuALfDG2jwYR443oOrufTQkpUA1s"  # RMI_Intentions_Test_Dataset.json

print("="*60)
print("LOADING FILES FROM GOOGLE DRIVE")
print("="*60)

# Descargar outputs generados
print("\nDownloading generated outputs...")
generated_content = download_file_from_drive(generated_outputs_file_id, "generated_tmf_format_outputs.json")
if generated_content:
    try:
        tmf_format_outputs = json.loads(generated_content)
        print(f"  ✓ Loaded {len(tmf_format_outputs)} generated outputs")
    except json.JSONDecodeError as e:
        print(f"  ✗ Error parsing generated outputs JSON: {e}")
        raise
else:
    print("  ✗ Failed to load generated outputs")
    raise Exception("Cannot continue without generated outputs")

# Descargar dataset de evaluación
print("\nDownloading evaluation dataset...")
eval_content = download_file_from_drive(eval_dataset_file_id, "RMI_Intentions_Test_Dataset.json")
if eval_content:
    try:
        eval_json_dataset = json.loads(eval_content)
        print(f"  ✓ Loaded evaluation dataset")
        print(f"  Dataset structure: {type(eval_json_dataset)}")
        if isinstance(eval_json_dataset, dict):
            print(f"  Dataset keys: {list(eval_json_dataset.keys())}")
    except json.JSONDecodeError as e:
        print(f"  ✗ Error parsing evaluation dataset JSON: {e}")
        raise
else:
    print("  ✗ Failed to load evaluation dataset")
    raise Exception("Cannot continue without evaluation dataset")

# ============================================================================
# 4. EXTRAER RESPUESTAS DE REFERENCIA (ESTRUCTURA CORREGIDA)
# ============================================================================

print("\nExtracting reference answers...")
reference_outputs = []

# El dataset tiene la estructura: {"conversations": [[{msg1}, {msg2}], [{msg1}, {msg2}], ...]}
# Cada elemento en la lista es una conversación completa
if isinstance(eval_json_dataset, dict) and 'conversations' in eval_json_dataset:
    conversations_list = eval_json_dataset['conversations']
    print(f"  Found {len(conversations_list)} conversations")

    for i, conversation in enumerate(conversations_list):
        # Cada conversación es una lista de mensajes
        for message in conversation:
            if message.get('from') == 'model_answer':
                # Extraer el texto dentro de los triple backticks
                answer = message.get('value', '')

                # Buscar contenido dentro de triple backticks usando regex
                import re
                pattern = re.search(r"```(.*?)```", answer, re.DOTALL)
                if pattern:
                    extracted_text = pattern.group(1).strip()
                    reference_outputs.append(extracted_text)
                else:
                    # Si no hay backticks, usar el valor completo
                    reference_outputs.append(answer)
                break  # Solo tomar la primera respuesta del modelo en la conversación

    print(f"  ✓ Extracted {len(reference_outputs)} reference answers")
else:
    print("  ✗ Unexpected dataset structure")
    raise Exception("Dataset does not have expected 'conversations' key")

# Verificar que tengamos el mismo número de outputs y referencias
print(f"\nData alignment check:")
print(f"  Generated outputs: {len(tmf_format_outputs)}")
print(f"  Reference answers: {len(reference_outputs)}")

if len(tmf_format_outputs) != len(reference_outputs):
    print(f"  ⚠ WARNING: Different counts - adjusting to minimum")
    min_len = min(len(tmf_format_outputs), len(reference_outputs))
    tmf_format_outputs = tmf_format_outputs[:min_len]
    reference_outputs = reference_outputs[:min_len]
    print(f"  ✓ Adjusted to {min_len} samples")
else:
    print(f"  ✓ Counts match perfectly")

# ============================================================================
# 5. CALCULAR BLEU SCORE
# ============================================================================

def calculate_bleu_scores(generated_outputs, reference_outputs):
    """Calcula BLEU score para cada par de salidas generadas y referencias."""
    bleu_scores = []
    smooth = SmoothingFunction()

    print("\n" + "="*60)
    print("CALCULATING BLEU SCORES...")
    print("="*60)

    for i, (generated, reference) in enumerate(zip(generated_outputs, reference_outputs)):
        # Manejar casos donde generated o reference puedan estar vacíos
        if not generated or not reference:
            print(f"\n⚠ Sample {i+1}: Empty generated or reference text, skipping...")
            bleu_scores.append(0.0)
            continue

        # Tokenizar
        generated_tokens = generated.split()
        reference_tokens = [reference.split()]

        # Calcular BLEU con suavizado
        score = sentence_bleu(
            reference_tokens,
            generated_tokens,
            smoothing_function=smooth.method1
        )
        bleu_scores.append(score)

        if i < 3:  # Mostrar primeros 3 ejemplos
            print(f"\nSample {i+1}:")
            print(f"  Generated (first 150 chars): {generated[:150]}...")
            print(f"  Reference (first 150 chars): {reference[:150]}...")
            print(f"  BLEU Score: {score:.4f}")

    # Calcular corpus BLEU usando sacrebleu
    corpus_bleu = sacrebleu.corpus_bleu(generated_outputs, [reference_outputs])

    results = {
        'individual_scores': bleu_scores,
        'average_bleu': np.mean(bleu_scores),
        'corpus_bleu': corpus_bleu.score,
        'min_bleu': np.min(bleu_scores),
        'max_bleu': np.max(bleu_scores),
        'std_bleu': np.std(bleu_scores)
    }

    print("\n" + "-"*60)
    print("BLEU SCORE RESULTS:")
    print("-"*60)
    print(f"Average BLEU:     {results['average_bleu']:.4f}")
    print(f"Corpus BLEU:      {results['corpus_bleu']:.4f}")
    print(f"Min BLEU:         {results['min_bleu']:.4f}")
    print(f"Max BLEU:         {results['max_bleu']:.4f}")
    print(f"Std Deviation:    {results['std_bleu']:.4f}")
    print("-"*60)

    return results

# Calcular BLEU
bleu_results = calculate_bleu_scores(tmf_format_outputs, reference_outputs)

# ============================================================================
# 6. CALCULAR ROC-AUC (BASADO EN SIMILITUD)
# ============================================================================

def calculate_roc_auc_similarity(generated_outputs, reference_outputs):
    """Calcula ROC-AUC usando similitud de texto como proxy."""

    print("\n" + "="*60)
    print("CALCULATING ROC-AUC (SIMILARITY-BASED)...")
    print("="*60)
    print("Note: ROC-AUC adapted for text generation using similarity scores")

    # Calcular similitudes
    similarities = []
    y_true = []

    for i, (gen, ref) in enumerate(zip(generated_outputs, reference_outputs)):
        if not gen or not ref:
            similarities.append(0.0)
            y_true.append(1)
            continue

        similarity = SequenceMatcher(None, gen, ref).ratio()
        similarities.append(similarity)
        y_true.append(1)  # Siempre 1 (positivo) para la referencia correcta

        if i < 3:
            print(f"\nSample {i+1} Similarity: {similarity:.4f}")

    # Crear predicciones binarias basadas en threshold
    threshold = 0.5
    y_pred_binary = [1 if s >= threshold else 0 for s in similarities]

    # Calcular métricas
    try:
        # ROC-AUC usando similitudes como scores
        roc_auc = roc_auc_score(y_true, similarities)

        # Calcular curva ROC
        fpr, tpr, thresholds = roc_curve(y_true, similarities)

        # Graficar curva ROC
        plt.figure(figsize=(10, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2,
                 label=f'ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate', fontsize=12)
        plt.ylabel('True Positive Rate', fontsize=12)
        plt.title('ROC Curve - Text Similarity Classification', fontsize=14, fontweight='bold')
        plt.legend(loc="lower right", fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('roc_curve.png', dpi=300, bbox_inches='tight')
        print("\n✓ ROC curve saved as 'roc_curve.png'")
        plt.close()

        # Calcular accuracy basada en threshold
        accuracy = sum([1 for p, t in zip(y_pred_binary, y_true) if p == t]) / len(y_true)

        results = {
            'roc_auc': roc_auc,
            'average_similarity': np.mean(similarities),
            'min_similarity': np.min(similarities),
            'max_similarity': np.max(similarities),
            'std_similarity': np.std(similarities),
            'accuracy_at_threshold': accuracy,
            'threshold_used': threshold,
            'individual_similarities': similarities
        }

        print("\n" + "-"*60)
        print("ROC-AUC RESULTS:")
        print("-"*60)
        print(f"ROC-AUC Score:           {roc_auc:.4f}")
        print(f"Average Similarity:      {results['average_similarity']:.4f}")
        print(f"Min Similarity:          {results['min_similarity']:.4f}")
        print(f"Max Similarity:          {results['max_similarity']:.4f}")
        print(f"Std Similarity:          {results['std_similarity']:.4f}")
        print(f"Accuracy (threshold={threshold}): {accuracy:.4f}")
        print("-"*60)

        return results

    except Exception as e:
        print(f"✗ Error calculating ROC-AUC: {e}")
        return None

# Calcular ROC-AUC
roc_results = calculate_roc_auc_similarity(tmf_format_outputs, reference_outputs)

# ============================================================================
# 7. GUARDAR RESULTADOS
# ============================================================================

from datetime import datetime

# Timestamp
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Crear diccionario con todos los resultados
metrics_results = {
    'timestamp': timestamp,
    'total_samples': len(tmf_format_outputs),
    'bleu_metrics': {
        'average_bleu': float(bleu_results['average_bleu']),
        'corpus_bleu': float(bleu_results['corpus_bleu']),
        'min_bleu': float(bleu_results['min_bleu']),
        'max_bleu': float(bleu_results['max_bleu']),
        'std_bleu': float(bleu_results['std_bleu']),
        'individual_scores': [float(s) for s in bleu_results['individual_scores']]
    },
    'roc_auc_metrics': {
        'roc_auc': float(roc_results['roc_auc']) if roc_results else None,
        'average_similarity': float(roc_results['average_similarity']) if roc_results else None,
        'min_similarity': float(roc_results['min_similarity']) if roc_results else None,
        'max_similarity': float(roc_results['max_similarity']) if roc_results else None,
        'std_similarity': float(roc_results['std_similarity']) if roc_results else None,
        'accuracy_at_threshold': float(roc_results['accuracy_at_threshold']) if roc_results else None,
        'individual_similarities': [float(s) for s in roc_results['individual_similarities']] if roc_results else []
    }
}

# Guardar como JSON localmente
output_filename = f'metrics_results_{timestamp}.json'
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(metrics_results, f, indent=2, ensure_ascii=False)

print("\n" + "="*60)
print("RESULTS SAVED LOCALLY")
print("="*60)
print(f"✓ Metrics saved to: {output_filename}")
print(f"✓ ROC curve saved to: roc_curve.png")
print("="*60)

# Resumen final
print("\n" + "="*60)
print("EVALUATION SUMMARY")
print("="*60)
print(f"Total samples evaluated: {len(tmf_format_outputs)}")
print(f"\nBLEU Metrics:")
print(f"  Average BLEU Score: {bleu_results['average_bleu']:.4f}")
print(f"  Corpus BLEU Score:  {bleu_results['corpus_bleu']:.4f}")
if roc_results:
    print(f"\nROC-AUC Metrics:")
    print(f"  ROC-AUC Score:      {roc_results['roc_auc']:.4f}")
    print(f"  Avg Similarity:     {roc_results['average_similarity']:.4f}")
print("="*60)

Packages installed successfully
Authenticating with Google Drive...
✓ Authenticated successfully

LOADING FILES FROM GOOGLE DRIVE

  ✓ Loaded 15 generated outputs

  ✓ Loaded evaluation dataset
  Dataset structure: <class 'dict'>
  Dataset keys: ['conversations']

Extracting reference answers...
  Found 10 conversations
  ✓ Extracted 10 reference answers

Data alignment check:
  Generated outputs: 15
  Reference answers: 10
  ⚠ WARNING: Different counts - adjusting to minimum
  ✓ Adjusted to 10 samples

CALCULATING BLEU SCORES...

⚠ Sample 1: Empty generated or reference text, skipping...

⚠ Sample 2: Empty generated or reference text, skipping...

⚠ Sample 3: Empty generated or reference text, skipping...

⚠ Sample 4: Empty generated or reference text, skipping...

⚠ Sample 5: Empty generated or reference text, skipping...

⚠ Sample 8: Empty generated or reference text, skipping...

⚠ Sample 9: Empty generated or reference text, skipping...

⚠ Sample 10: Empty generated or reference t

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:379: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_ranking.py:1179: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(



✓ ROC curve saved as 'roc_curve.png'

------------------------------------------------------------
ROC-AUC RESULTS:
------------------------------------------------------------
ROC-AUC Score:           nan
Average Similarity:      0.0458
Min Similarity:          0.0000
Max Similarity:          0.2664
Std Similarity:          0.0931
Accuracy (threshold=0.5): 0.0000
------------------------------------------------------------

RESULTS SAVED LOCALLY
✓ Metrics saved to: metrics_results_2025-10-03_03-59-12.json
✓ ROC curve saved to: roc_curve.png

EVALUATION SUMMARY
Total samples evaluated: 10

BLEU Metrics:
  Average BLEU Score: 0.0227
  Corpus BLEU Score:  0.0364

ROC-AUC Metrics:
  ROC-AUC Score:      nan
  Avg Similarity:     0.0458
